In [2]:
from __future__ import print_function
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.utils import np_utils

Using TensorFlow backend.
C:\Users\karth\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\karth\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\karth\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\karth\AppData\Roa

In [25]:
test_dim = 999
maxlen = 100
batch_size = 100
nb_filter = 64
filter_length_1 = 50
filter_length_2 = 25
hidden_dims = 250
nb_epoch = 8
nb_classes = 10

In [10]:
X = np.load('X_all.npy')
y = np.load('y_all.npy')

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [60]:
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

1710 428
1710 428


In [42]:

# xts = X_train.shape
# X_train = np.reshape(X_train, (xts[0], xts[1],xts[2],1))
# xtss = X_test.shape
# X_test = np.reshape(X_test, (xtss[0], xtss[1],xts[2],1))
# yts = y_train.shape
# y_train = np.reshape(y_train, (yts[0], 1))
# ytss = y_test.shape
# y_test = np.reshape(y_test, (ytss[0], 1))

In [69]:
X_train[0].shape

(20, 85)

In [65]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [66]:
print(len(Y_train), len(Y_test))

1710 428


In [67]:
Y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [79]:
model = Sequential()
model.add(Convolution1D(nb_filter=64,filter_length=3,
                        input_shape=(20,85),
                        
                        border_mode='valid',
                        activation='relu'
                        ))
# we use standard max pooling (halving the output of the previous layer):
model.add(BatchNormalization())

model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length_2,
                        border_mode='same',
                        activation='relu'
                        ))

model.add(BatchNormalization())

model.add(MaxPooling1D(pool_length=2))

model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length_2,
                        border_mode='same',
                        activation='relu'
                        ))

model.add(BatchNormalization())

model.add(MaxPooling1D(pool_length=2))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
# model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
# model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
               metrics=['accuracy'])
# model.summary()

model.fit(X_train, Y_train, batch_size=batch_size,
          nb_epoch=nb_epoch,  verbose=1,
          validation_data=(X_test, Y_test))

y_preds = model.predict(X_test)

score = model.evaluate(X_test, Y_test,  verbose=1)

print("score",score)



C:\Users\karth\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(20, 85), activation="relu", filters=64, kernel_size=3, padding="valid")`
  
C:\Users\karth\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=25, padding="same")`
  
C:\Users\karth\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
C:\Users\karth\AppData\Local\Continuum\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=64, kernel_size=25, padding="same")`
C:\Users\karth\AppData\Local\Continuum\anaconda3\envs\pytorch\l

Train on 1710 samples, validate on 428 samples
Epoch 1/8
1710/1710 [==============================] - 4s 2ms/step - loss: 0.3088 - accuracy: 0.8944 - val_loss: 0.3267 - val_accuracy: 0.9037
Epoch 2/8
1710/1710 [==============================] - 1s 598us/step - loss: 0.2745 - accuracy: 0.8989 - val_loss: 0.3130 - val_accuracy: 0.9037
Epoch 3/8
1710/1710 [==============================] - 1s 623us/step - loss: 0.2686 - accuracy: 0.9006 - val_loss: 0.2869 - val_accuracy: 0.9037
Epoch 4/8
1710/1710 [==============================] - 1s 577us/step - loss: 0.2688 - accuracy: 0.8992 - val_loss: 0.2771 - val_accuracy: 0.9037
Epoch 5/8
1710/1710 [==============================] - 1s 640us/step - loss: 0.2665 - accuracy: 0.8998 - val_loss: 0.2619 - val_accuracy: 0.9028
Epoch 6/8
1710/1710 [==============================] - 1s 589us/step - loss: 0.2672 - accuracy: 0.8987 - val_loss: 0.2617 - val_accuracy: 0.9037
Epoch 7/8
1710/1710 [==============================] - 1s 577us/step - loss: 0.2681 -